## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-18-16-48-40 +0000,bbc,US accused of using illegal workers at centre ...,https://www.bbc.com/news/articles/c2468q3y75vo...
1,2025-12-18-16-47-18 +0000,nyt,Massachusetts Battery Start-Up to List on Stoc...,https://www.nytimes.com/2025/12/18/business/en...
2,2025-12-18-16-46-22 +0000,bbc,Zelensky gives stark warning as EU leaders hol...,https://www.bbc.com/news/articles/c9vjlj1ezpgo...
3,2025-12-18-16-45-56 +0000,nyt,Iran’s President Says He Can’t Make ‘Miracles’...,https://www.nytimes.com/2025/12/18/world/middl...
4,2025-12-18-16-45-22 +0000,nyt,The D.N.C. Is Scrapping Its Report on What Wen...,https://www.nytimes.com/2025/12/18/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
87,trump,60
138,new,22
311,bondi,15
350,reiner,14
549,house,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
302,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...,155
336,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...,144
320,2025-12-17-20-48-00 +0000,wsj,The Senate passed the sweeping defense-policy ...,https://www.wsj.com/politics/national-security...,110
190,2025-12-18-06-03-00 +0000,wsj,President Trump defended his handling of the e...,https://www.wsj.com/politics/policy/trump-defe...,108
248,2025-12-18-00-20-46 +0000,cbc,Trump trade rep targets Canada's beer and dair...,https://www.cbc.ca/news/politics/trump-cusma-c...,106


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
302,155,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...
336,77,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...
132,69,2025-12-18-11-09-02 +0000,nypost,Cardinal Dolan emotionally welcomes Illinois B...,https://nypost.com/2025/12/18/us-news/pope-leo...
340,58,2025-12-17-19-55-55 +0000,nypost,Top cop defends Bondi Beach officer accused of...,https://nypost.com/2025/12/17/world-news/top-c...
51,54,2025-12-18-14-55-00 +0000,wsj,The Bank of England cut its key interest rate ...,https://www.wsj.com/economy/central-banking/ba...
326,52,2025-12-17-20-36-00 +0000,wsj,FCC Chair Brendan Carr told a Senate oversight...,https://www.wsj.com/politics/policy/fcc-isnt-a...
281,48,2025-12-17-22-45-06 +0000,bbc,Rob Reiner's family breaks silence as son Nick...,https://www.bbc.com/news/articles/cx239pp7gl7o...
48,45,2025-12-18-15-03-00 +0000,wsj,Oil Extends Gains With Market Watching Venezue...,https://www.wsj.com/finance/commodities-future...
135,44,2025-12-18-11-00-00 +0000,nypost,Unseen photo reveals time Bill Clinton demande...,https://nypost.com/2025/12/18/us-news/bill-cli...
13,43,2025-12-18-16-28-59 +0000,nyt,"Trump Media to Merge With TAE Technologies, a ...",https://www.nytimes.com/2025/12/18/business/tr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
